## 과제

In [52]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.8.2


In [53]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [54]:
url = "https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1"
driver.get(url)
driver.maximize_window() # 브라우저의 크기 전체화면으로 확대

In [75]:
codi_list = []

## 페이지를 위부터 아래로 스크롤 해보자.
for i in range(1):
    ## 1초에 한번씩 스크롤 내리기.
    interval = 1
    ## 현재 문서 높이 저장.
    prev_height = driver.execute_script("return document.body.scrollHeight")
    ## prev_height가 while loop 안의 curr_height와 같으면 스크롤을 끝까지 내렸다는 뜻.
    while True:
        ## 스크롤 내리기
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        ## 페이지 로딩 대기.
        time.sleep(interval)
        ## 현재 문서 높이 저장.
        curr_height = driver.execute_script("return document.body.scrollHeight")
        if curr_height == prev_height:
            break
    
    ## 카데고리, 제목, 날짜, 조회수, 댓글 수
    columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers']
    ## 위의 정보를 담을 빈 리스트.
    values = []
    ## 현재 브라우저 정보 가져오기.
    soup = BeautifulSoup(driver.page_source,'lxml')
    
    ## 아이템 하나에 대한 값을 모두 포함하는 큰 항목이 총 60개 존재. 먼저 이 녀석들 데이터를 통째로 긁어온다.
    data_box = soup.find_all('li',attrs = {'class':'style-list-item'})
    
    for index, info in enumerate(data_box):
    
        print('{}번째 코디정보 크롤링'.format(index+1))
        blank = []
    
        codimap_category = info.find('span',attrs={'class':'style-list-information__text'})
        if codimap_category:
            codimap_category = codimap_category.get_text().strip()
            blank.append(codimap_category)
        else:
            blank.append('Something is wrong')
            print('{}번째 코디 카테고리 가져올때 문제발생'.format(i+1))
            continue
    
        codimap_title = info.find('strong',attrs={'class':'style-list-information__title'})
        if codimap_title:
            codimap_title = codimap_title.get_text().strip()
            blank.append(codimap_title)
        else:
            blank.append('Something is wrong')
            print('{}번째 코디 제목 가져올때 문제발생'.format(i+1))
            continue
    
        codimap_date, views, comment_numbers = info.find_all('span','post-information__text')
        if codimap_date and views and comment_numbers:
            codimap_date = codimap_date.get_text().strip()
            views = views.get_text().strip()[3:]
            comment_numbers = comment_numbers.get_text().strip()[3:]
            blank.append(codimap_date)
            blank.append(views)
            blank.append(comment_numbers)
        else:
            blank.append('Something is wrong')
            print('{}번째 코디 날짜/조회수/댓글 수 가져올 때 문제발생'.format(i+1))
            continue
            
        values.append(blank)
        print("----------------------------------------------------------------")
        
    df = pd.DataFrame(values, columns = columns)
    codi_list.append(df)
    

1번째 코디정보 크롤링
----------------------------------------------------------------
2번째 코디정보 크롤링
----------------------------------------------------------------
3번째 코디정보 크롤링
----------------------------------------------------------------
4번째 코디정보 크롤링
----------------------------------------------------------------
5번째 코디정보 크롤링
----------------------------------------------------------------
6번째 코디정보 크롤링
----------------------------------------------------------------
7번째 코디정보 크롤링
----------------------------------------------------------------
8번째 코디정보 크롤링
----------------------------------------------------------------
9번째 코디정보 크롤링
----------------------------------------------------------------
10번째 코디정보 크롤링
----------------------------------------------------------------
11번째 코디정보 크롤링
----------------------------------------------------------------
12번째 코디정보 크롤링
----------------------------------------------------------------
13번째 코디정보 크롤링
-----------------------------------------------

In [76]:
df_ = pd.concat(codi_list).reset_index(drop=True) # index 초기화 해줘야함
df_.head(10)

,codimap_category,codimap_title,codimap_date,views,comment_numbers
0,캐주얼,톤온톤 코디,21.07.02,"16,049",49
1,캐주얼,블랙 매니아,21.02.23,"34,670",46
2,캐주얼,오늘의 코디,22.03.22,"12,198",38
3,댄디,추천 댄디 룩,21.09.15,"20,184",27
4,캐주얼,릴랙스 룩,21.02.23,"23,536",22
5,스트릿,힙하다 힙해,21.02.03,"35,286",21
6,캐주얼,심플한 코디,21.07.30,"39,574",18
7,댄디,데이트 코디,21.12.24,"10,309",18
8,댄디,훈훈 남친 룩,21.03.24,"19,343",16
9,캐주얼,지금 입기 좋아,21.08.31,"20,212",16


In [77]:
df.to_csv('C:/Users/wjdgh/Desktop/YBIGTA/Summer session/0707 Crawling/무신사_코디맵_크롤링.csv',encoding='utf-8-sig')